In [1]:
# pip install PublicDataReader --upgrade

     ---------------------------------------- 3.2/3.2 MB 29.1 MB/s eta 0:00:00
     ---------------------------------------- 3.9/3.9 MB 35.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Import Lib

In [59]:
from PublicDataReader import Kbland
import pandas as pd
import numpy as np

api = Kbland()

# PIR

In [26]:
params = {
    "메뉴코드": "01",
    "기간":"",
}
df = api.get_pir(**params)

In [32]:
df = df[['지역코드','지역명','주택분위','소득분위','날짜','PIR']]

In [33]:
df

,지역코드,지역명,주택분위,소득분위,날짜,PIR
0,0000000000,전국,5,5,2008-12-01,7.037341
1,1100000000,서울특별시,5,5,2008-12-01,11.537793
2,0000000000,전국,4,5,2008-12-01,3.705283
3,1100000000,서울특별시,4,5,2008-12-01,6.912031
4,0000000000,전국,3,5,2008-12-01,2.366151
...,...,...,...,...,...,...
8895,1100000000,서울특별시,3,1,2023-09-01,26.740611
8896,0000000000,전국,2,1,2023-09-01,7.706675
8897,1100000000,서울특별시,2,1,2023-09-01,15.840010
8898,0000000000,전국,1,1,2023-09-01,3.827658


# 주택구매력지수

In [34]:
params = {
    "기간": "1"
}
df = api.get_hai(**params)


In [37]:
df

,지역코드,지역명,날짜,종합,아파트,단독,연립
0,0000000000,전국,2022-09-01,82.706510,78.044688,107.433819,183.766866
1,0000000000,전국,2022-10-01,81.522580,79.192849,105.340494,180.215739
2,0000000000,전국,2022-11-01,113.963795,92.859388,129.682939,227.927590
3,0000000000,전국,2022-12-01,116.800547,95.297518,130.897165,230.061684
4,0000000000,전국,2023-01-01,123.796573,102.452336,136.603115,240.090324
...,...,...,...,...,...,...,...
99,4A0000,기타지방,2023-05-01,231.731594,210.522262,250.925524,552.036154
100,4A0000,기타지방,2023-06-01,230.305467,209.617010,249.846537,549.662382
101,4A0000,기타지방,2023-07-01,239.796957,218.368293,259.174488,570.183874
102,4A0000,기타지방,2023-08-01,239.179149,217.805693,258.506756,568.714864


# 가격지수1

In [73]:
building_category = ['01','08','09','98']
trade_category = ['01','02']

df_price_index = pd.DataFrame()
itercount = 0

for i in building_category:
    for j in trade_category:
        params = {
            "월간주간구분코드": "01",
            "매물종별구분": i,
            "매매전세코드": j,
            "지역코드": "",
            "기간": "",
        }
        df = api.get_price_index(**params)
        
        # api별로 컬럼명 변경을 위한 단어추출
        first_word = df[df.columns[:3]].iloc[0].values[0]
        second_word = df[df.columns[:3]].iloc[0].values[1]
        third_word = df[df.columns[:3]].iloc[0].values[2]
        last_word = df.columns[-1]
        new_col_name = first_word + ' ' + second_word + ' ' + third_word + ' ' + last_word

        # 지수컬럼명 변경
        df = df.rename(columns = {
                    last_word : new_col_name
                })

        # 필요한 컬럼만 남기기
        df = df[df.columns[3:]]
        
        if itercount == 0:
            df_price_index = pd.concat([df_price_index, df])
        else:
            df_price_index = pd.merge(df_price_index, df, on = ['지역코드','지역명','날짜'], how = 'left')
        
        itercount += 1

In [74]:
df_price_index

,지역코드,지역명,날짜,월간 아파트 매매 가격지수,월간 아파트 전세 가격지수,월간 연립 매매 가격지수,월간 연립 전세 가격지수,월간 단독 매매 가격지수,월간 단독 전세 가격지수,월간 주택종합 매매 가격지수,월간 주택종합 전세 가격지수
0,0000000000,전국,1986-01-01,15.910037,9.009376,37.418669,16.090665,56.672388,33.498810,27.682150,15.133296
1,0000000000,전국,1986-02-01,15.910037,9.269012,37.307469,16.175801,56.672388,33.710381,27.682150,15.350002
2,0000000000,전국,1986-03-01,15.880737,9.554612,37.140670,16.771751,56.825972,34.486143,27.723591,15.711179
3,0000000000,전국,1986-04-01,15.792836,9.762321,37.140670,17.069726,56.442012,35.050333,27.516389,16.036238
4,0000000000,전국,1986-05-01,15.675635,9.788285,36.362272,17.197430,56.211637,35.050333,27.392068,16.072355
...,...,...,...,...,...,...,...,...,...,...,...
10939,5100000000,강원,2023-08-01,101.957246,104.988239,103.249113,101.964600,100.989546,100.743040,100.710945,102.861530
10940,5100000000,강원,2023-09-01,102.189447,105.273099,103.249113,101.964600,100.989546,100.743040,100.896245,103.088450
10941,5100000000,강원,2023-10-01,102.332312,105.387976,103.249113,101.964600,100.989546,100.743040,101.009515,103.178796
10942,5100000000,강원,2023-11-01,102.491324,105.685756,103.249113,101.964600,100.989546,100.743040,101.136594,103.416384
